In [18]:
import numpy as np
from plotly import offline
from plotly import graph_objs as go
from PIL import Image
import plotly.express as px
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from skimage.filters import threshold_minimum
import torch
import os
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from tensorflow.keras import regularizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import tensorflow as tf

In [19]:
color = px.colors.sequential.Rainbow[::-1]
marker_size = 3

In [20]:
'''
    inputs :: 
        exp_name => polar arrays name which saved in 3 files like "exp_name_x.txt"
        folder_indx => choose the directory to load the arrays from it. We have 2 directories (0 => 3D_Experements, 1 => flat_Experements)
                                                                                                3D_Experements contains to positive experements
                                                                                                flat_Experements contains to nigative experements
        depth_axis => indicates the axis that we will colour based on it (x, y, z).
        threshold => will be applied on the depth axis and can get it from otsu's threshold.
    return ::
        NONE
    output :: 
        shows the 3d point cloud. 
'''
def plot_3D_threshold (exp_name ,folder_indx , depth_axis ,threshold):
    # load polar arrays 
    folder = ["3D_Experements", "flat_Experements"]
    dist = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_x.txt")
    upper_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_y.txt")
    lower_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_z.txt")
    
    print(np.amax(upper_angle*180/np.pi))
    print(np.amin(upper_angle*180/np.pi))
    print(np.amax(lower_angle*180/np.pi))
    print(np.amin(lower_angle*180/np.pi))
    
    # get cartesian coordinates from the polar coordinates.
    my_sample_x = np.array(dist)*np.cos(upper_angle)*np.sin(lower_angle)
    my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
    my_sample_z =  np.array(dist)*np.sin(upper_angle)
    my_sample_x = my_sample_x[~np.isnan(my_sample_x)]
    my_sample_y = my_sample_y[~np.isnan(my_sample_y)]
    my_sample_z = my_sample_z[~np.isnan(my_sample_z)]
    
    # dataframe creation 
    df = pd.DataFrame()
    
    indx = []
    min_depth = 0
    max_depth = 0
    # choosing the desired depth axis and applay the threshold on it and also determine the min & max depth on that axis
    if depth_axis == 'x':
        min_depth = np.amin(my_sample_x)
        max_depth = threshold#(min_depth + threshold)
        indx = my_sample_x<= threshold#(min_depth + threshold)
        df['Depth'] = my_sample_x[indx]

    elif depth_axis == 'y':
        min_depth = np.amin(my_sample_y)
        max_depth = threshold#(min_depth + threshold)
        indx = my_sample_y <= threshold#(min_depth + threshold)
#         print(my_sample_y , (min_depth + threshold))
#         print(type(my_sample_y),type(my_sample_y[indx]))
        df['Depth'] = my_sample_y[indx]

    elif depth_axis == 'z':
        min_depth = np.amin(my_sample_z)
        max_depth = threshold#(min_depth + threshold)
        indx = my_sample_z<= threshold#(min_depth + threshold)
        df['Depth'] = my_sample_z[indx]
    else:
        print("invalid depth axis")
        return
    df['X (mm)'] = my_sample_x[indx]
    df['Y (mm)'] = my_sample_y[indx]
    df['Z (mm)'] = my_sample_z[indx]
    df.head()
   
    # plot the 3D_point cloud
    # fig = px.scatter_3d(df, x='X (mm)', y='Y (mm)', z='Z (mm)', color='Depth', title="ٌRadar Point Cloud" , range_color=[min_depth,max_depth], range_y=[300,700],color_continuous_scale=color , opacity=1)
    fig = px.scatter_3d(df, x='X (mm)', y='Y (mm)', z='Z (mm)', color='Depth', title="ٌRadar Point Cloud" , range_color=[min_depth,max_depth],color_continuous_scale=color , opacity=1)
    fig.update_traces(marker=dict(size=marker_size, line=dict(width=0))) 
    fig.show()

In [21]:
plot_3D_threshold("Ball_40cm_100",1,'y',1000)

18.0
-18.0
18.0
-18.0


In [22]:
min_dist = 0
max_dist=800
bin_size = 4

In [37]:
'''
    inputs :: 
        exp_name => polar arrays name which saved in 3 files like "exp_name_x.txt"
        folder_indx => choose the directory to load the arrays from it. We have 2 directories (0 => 3D_Experements, 1 => flat_Experements)
                                                                                                3D_Experements contains to positive experements
                                                                                                flat_Experements contains to nigative experements
    return ::
        threshold => otsu's threshold.
        hist => histogram after applying otsu's threshold.
        bin_edges => these bin_edges will be determined based on the 3 variables in the above cell 
                        min_dist = 0
                        max_dist=800
                        bin_size = 4
                        so will be like (0, 4, 8,...., 800)
        hist_new => histogram after applying eqilization on the hist
        bin_edges_new => same as bin_edges
    output :: 
        NONE 
'''
def plot_hist(exp_name,folder_indx):
    # load polar arrays
    np.seterr(divide='ignore', invalid='ignore')
    folder = ["3D_Experements", "flat_Experements"]
    dist = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_x.txt")
    upper_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_y.txt")
    lower_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_z.txt")
    
    # get cartesian coordinates from the polar coordinates.
    my_sample_y = np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
    
    my_sample_y = my_sample_y[~np.isnan(my_sample_y)]
    
    # otsu's calculations 
    # Set total number of bins in the histogram
    bins_num = [i for i in range(min_dist,max_dist,bin_size)]#256
    n = np.histogram(my_sample_y , bins = bins_num) 
#     n = plt.hist(my_sample_y , bins = bins_num) 
    hist = n[0]
    bin_edges = n[1]  
    # Calculate centers of bins
    bin_mids = (bin_edges[:-1] + bin_edges[1:]) / 2.
    # Iterate over all thresholds (indices) and get the probabilities w1(t), w2(t)
    weight1 = np.cumsum(hist)
    weight2 = np.cumsum(hist[::-1])[::-1]
    # Get the class means mu0(t)
    mean1 = np.cumsum(hist * bin_mids) / weight1
    # Get the class means mu1(t)
    mean2 = (np.cumsum((hist * bin_mids)[::-1]) / weight2[::-1])[::-1]
    mean1 = np.nan_to_num(mean1)
    mean2 = np.nan_to_num(mean2)
    inter_class_variance = weight1[:-1] * weight2[1:] * (mean1[:-1] - mean2[1:]) ** 2
    # Maximize the inter_class_variance function val
    index_of_max_val = np.argmax(inter_class_variance)
    threshold = bin_mids[:-1][index_of_max_val]
#     print("Otsu's algorithm implementation thresholding result: ", threshold)
#     plt.title("histogram") 
#     plt.axvline(threshold, color='r')
#     plt.show()
    # applay otsu's threshold on the histogram 
    hist = [ hist[i] if bin_edges[i] < threshold else 0 for i in range(hist.size)]
    my_sample_y = my_sample_y[my_sample_y < threshold]
    # Histogram Equalization
    total_value = np.sum(hist)
    hist_prob = hist / total_value
    hist_cumsum = np.cumsum(hist_prob)
    new_hist = hist_cumsum * 800
    new_hist = np.floor(new_hist).astype(int)
    new_my_sample_y = np.zeros(len(my_sample_y))
    for idx in range (len(my_sample_y)):
        for i in range(len(bin_edges)):
            if bin_edges[i]> my_sample_y[idx] :
                new_my_sample_y[idx] = new_hist[i-1]
                break
    n = np.histogram(new_my_sample_y , bins = bins_num) 
    hist_new = n[0]
    bin_edges_new = n[1]
#     x = my_sample_y
#     y = new_my_sample_y
#     bins = bins_num
#     plt.hist(x, bins_num, alpha=0.5, label='x')
#     plt.hist(y, bins_num, alpha=0.5, label='y')
#     plt.legend(loc='upper right')
#     plt.show()
    return threshold,hist,bin_edges,hist_new,bin_edges_new

In [24]:
t , h, e ,h_n , e_n = plot_hist("test_book_100",1)

# SVM Method

In [ ]:
############################################## SVM ###################################################################
'''
    This function is used to create separated dataset files for each experement
''' 
def create_dataset():

    exp_p = [
        # "waleed_60CM", 
        # "nassar_50cm",
        # "gedo_50cm",
        # "gedo_14_7",
        # "waleed_14_7",
        # "ta7a_14_7",
        # "nassar_14_7"
        # "nassar_mask_glasses"
        # "gedo_mask_glass"
        # "gedo_bro2",
        # "gedo_sis2",
        # "gedo_sis1",
        # "gedo_sis1_headphone",
        # "gedo_mum",
        # "gedo_bro1_mask_glass",
        # "gedo_bro1",
        # "waleed_noisy",
        # "waleed_bad_ex"
        # "ahmed_mum_40cm",
        # "ahmed_mum2"
        # "nassar_mask"
    ]

    exp_n = [
        # "painting_50cm_rotation2",
        # "painting_50cm_rotation",
        # "painting_50cm",
        # "painting_40cm_with_angle",
        # "painting_40cm",
        # "metal_dish",
        # "painting", 
        # "ahmed_image_40cm", 
        # "ahmed_image_40cm_with_chair_behind", 
        # "ahmed_image_50cm", 
        # "ahmed_image_50cm_with_chair_behind", 
        # "ahmed_image_60cm", 
        # "ahmed_image_60cm_with_chair_behind", 
        # "Ball_40cm", 
        # "Ball_50cm", 
        # "Ball_60cm", 
        # "basket_40cm", 
        # "basket_50cm", 
        # "basket_60cm", 
        # "vas",
        # "air",
        # "noisy_air",
        # "wall_22_7",
        # "test_book",
        # "test_wall_23_7",
        # "test_chair",
        # "test_book_50cm",
        # "old_laptop",
        # "old_laptop_40cm",
    ]
    count = 0

    for i in range(len(exp_n)):
        feature_hist = []
        target_list = []
        for j in range(1000):
            exp_name = exp_n[i] + "_" + str(j)
            t , h ,e , h_n, e_n= plot_hist(exp_name,1)
            feature_hist.append(h_n)
            target_list.append(-1)
            count += 1
            print("count :: ",count)
        feature_hist = np.array(feature_hist)
        target_list = np.array(target_list)
        
        file_data = open("equalized_feature/"+exp_n[i]+"_features.txt", "a")
        np.savetxt(file_data, feature_hist)
        file_data.close()

        file_target_list = open("equalized_feature/"+exp_n[i]+"_target_list.txt", "a")
        np.savetxt(file_target_list, target_list)
        file_target_list.close()
        
    for i in range(len(exp_p)):
        feature_hist = []
        target_list = []
        for j in range(1000):
            exp_name = exp_p[i] + "_" + str(j)
            t , h ,e ,h_n , e_n = plot_hist(exp_name,0)
            feature_hist.append(h_n)
            target_list.append(1)
            count += 1
            print("count :: ",count)
        feature_hist = np.array(feature_hist)
        target_list = np.array(target_list)
        
        file_data = open("equalized_feature/"+exp_p[i]+"_features.txt", "a")
        np.savetxt(file_data, feature_hist)
        file_data.close()

        file_target_list = open("equalized_feature/"+exp_p[i]+"_target_list.txt", "a")
        np.savetxt(file_target_list, target_list)
        file_target_list.close()
    return "done"

In [ ]:
############################################## SVM ###################################################################
print(create_dataset())

In [ ]:
############################################## SVM ###################################################################
'''
    Apply normalization step on the loaded features.
'''
def pre_processing(exp_name):
    target_list = np.loadtxt("equalized_feature/"+exp_name+"_target_list.txt")
    data_list = np.loadtxt("equalized_feature/"+exp_name+"_features.txt")
    for j in range(1000):
        data_list[j] = (data_list[j] - np.min(data_list[j])) / (np.max(data_list[j]) - np.min(data_list[j]))
    print(np.sum(data_list))
    return data_list,target_list

'''
    loading data and apply preprsessing on it and create one array that contains all data 
    then shuffle the data to load it to the model
'''
exp_p = [ 
        "gedo_14_7",
        "waleed_14_7",
        "ta7a_14_7",
        "nassar_14_7",
        "waleed_60CM",
        "gedo_50cm",
        "nassar_50cm",
        "waleed_mask_glass",
        "nassar_mask",
        "gedo_mask_glass",
        "nassar_mask_glasses",
        "gedo_bro1_mask_glass",
        "gedo_bro1",
        "gedo_bro2",
        "gedo_mum",
        "gedo_sis1",
        "gedo_sis1_headphone",
        "gedo_sis2",
        "waleed_noisy",
        "waleed_bad_ex",
        "ahmed_mum_40cm",
        "ahmed_mum2"
    ]

exp_n = [
        "metal_dish",
        "painting", 
        "ahmed_image_40cm", 
        "ahmed_image_40cm_with_chair_behind", 
        "ahmed_image_50cm", 
        "ahmed_image_50cm_with_chair_behind", 
        "ahmed_image_60cm", 
        "ahmed_image_60cm_with_chair_behind",
        "air",
        "painting_50cm_rotation2",
        "painting_50cm_rotation",
        "painting_50cm",
        "painting_40cm_with_angle",
        "painting_40cm",
        "noisy_air",
        "wall_22_7",
        "test_book",
        "test_wall_23_7",
        "test_chair",
        "strange_wall",
        "semi_flat_wood",
        "flat_wood",
        "flat_wall_60cm",
        "flat_wall_50cm",
        "flat_chair",
#         "test_book_50cm",
#     "old_laptop",
#     "old_laptop_40cm",
    ]
         
total_data = []
total_target_lists = []
for i in range(len(exp_n)):     
    data_list , target_list = pre_processing(exp_n[i])  
    target_list = target_list * 0 
    target_list = target_list.tolist()
    data_list = data_list.tolist()
    total_data.extend(data_list)
    total_target_lists.extend(target_list)
    
for i in range(len(exp_p)):
    data_list , target_list = pre_processing(exp_p[i])   
    target_list = target_list.tolist()
    data_list = data_list.tolist()
    total_data.extend(data_list)
    total_target_lists.extend(target_list)
    
total_data = np.array(total_data)
total_target_lists = np.array(total_target_lists)
total_target_lists = np.reshape(total_target_lists , (-1,1))
total_data = np.concatenate((total_data, total_target_lists), axis=1)
print(total_data[0])
np.random.shuffle(total_data)

print(total_data[0])


print("done")

In [ ]:
############################################## SVM ###################################################################
# Import train_test_split function
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(total_data[:,0:199], total_data[:,199], train_size=0.65,test_size=0.35, random_state=101) # 70% training and 30% test
# print (X_train)
#Create a svm Classifier



clf = svm.SVC(kernel='rbf' ) # Linear Kernel

#Train the model using the training sets
print (clf.fit(X_train, y_train).score(X_train, y_train))

#Predict the response for test dataset
# print (X_test.shape)
y_pred = clf.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
############################################## SVM ###################################################################
############################################## Save SVM Model ########################################################
from joblib import dump, load
dump(clf, 'SVM/rbf_model.pkl')

In [ ]:
############################################## SVM ###################################################################
############################################## Load SVM Model ########################################################
from joblib import dump, load
clf = load('SVM/rbf_model.pkl') 

In [ ]:
############################################## SVM ###################################################################
t , h, e , h_n , e_n = plot_hist("painting_50cm_rotation_100",1)
f = (h_n - np.min(h_n)) /(np.max(h_n) - np.min(h_n))
data_list = np.array([f])
y_pred = clf.predict(data_list)
print(y_pred)

In [ ]:
############################################## SVM ###################################################################
result_p = []
result_n = []
count = 0
for i in range(1000):
    t , h, e , h_n , e_n = plot_hist("old_laptop_40cm_"+str(i),1)
    # print(t,e,h.size)
    # f = [ h[n] if e[n] < t else 0 for n in range(h.size)]
    # print(f)

    # f =  100 *(f / np.sum(f))
    # print(f)
    f = (h_n - np.min(h_n)) /(np.max(h_n) - np.min(h_n))
    # f = np.trim_zeros(f)
    # # print(f)
    
    # N = 59 - len(f)
    # f = np.pad(f, (0, N), 'constant')
    # print(f)
    data_list = np.array([f])
    y_pred = clf.predict(data_list)
#     y_pred =nn_pred(data_list)
    count +=1
    print("count :: ",count)
    if y_pred == True or y_pred == 1:
        result_p.append(y_pred)
    else:
        result_n.append(y_pred)
        
        
print("number of positive results :: ", len(result_p))
print("number of negative results :: ", len(result_n))
print("positive accuracy :: ", (100 *len(result_p)/(len(result_p)+len(result_n)) ))
print("negative accuracy :: ", (100 *len(result_n)/(len(result_p)+len(result_n)) ))

# CNN Method

In [28]:
def points_to_image(exp_name,folder_indx,threshold):
    np.seterr(divide='ignore', invalid='ignore')
    folder = ["3D_Experements", "flat_Experements"]
    dist = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_x.txt")
    upper_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_y.txt")
    lower_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_z.txt")

#     print(np.amax(upper_angle*180/np.pi))
#     print(np.amin(upper_angle*180/np.pi))
#     print(np.amax(lower_angle*180/np.pi))
#     print(np.amin(lower_angle*180/np.pi))
    

    #x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)

    my_sample_x = np.array(dist)*np.cos(upper_angle)*np.sin(lower_angle)
    my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
    my_sample_z =  np.array(dist)*np.sin(upper_angle)
    
    my_sample_x = my_sample_x[~np.isnan(my_sample_x)]
    my_sample_y = my_sample_y[~np.isnan(my_sample_y)]
    my_sample_z = my_sample_z[~np.isnan(my_sample_z)]

    
    min_depth = np.amin(my_sample_y)
    max_depth = (min_depth + threshold)
    indx = my_sample_y <= (min_depth + threshold)

    my_sample_x = my_sample_x[indx]
    my_sample_y = my_sample_y[indx]
    my_sample_z = my_sample_z[indx]
    my_sample_y = 1-  (my_sample_y -  np.min(my_sample_y)) / (np.max(my_sample_y) - np.min(my_sample_y))
    x_min = np.min(my_sample_x)
    x_max = np.max(my_sample_x)
    y_min = np.min(my_sample_y)
    y_max = np.max(my_sample_y)
    z_min = np.min(my_sample_z)
    z_max = np.max(my_sample_z)
    width = 1+(x_max - x_min).astype(int)
    hight = 1+(z_max - z_min).astype(int)

    # my_sample_y = my_sample_y + (1-y_max)
#     print("img :: ",width,hight)
    img = np.zeros((width,hight))
    
    for i in range(len(my_sample_x)):
        x = (my_sample_x[i] - x_min).astype(int)
        z = (my_sample_z[i] - z_min).astype(int)
#         print("x,z :: ",x,z)
        img[x,z] = my_sample_y[i]
#     print('Resized Dimensions : ',resized.shape)

#     cv2.imshow("Resized image", resized)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     img = Image.fromarray(img)
#     print(len(my_sample_x))
    
    kernel = np.ones((5,5), np.uint8)
#     # The first parameter is the original image,
#     # kernel is the matrix with which image is
#     # convolved and third parameter is the number
#     # of iterations, which will determine how much
#     # you want to erode/dilate a given image.
#     img_erosion = cv2.erode(img, kernel, iterations=1)
    img_dilation = cv2.dilate(img, kernel, iterations=3)

#     cv2.imshow('Input', img)
#     cv2.imshow('Erosion', img_erosion)
#     cv2.imshow('Dilation', img_dilation)

#     cv2.waitKey(0)
#     img = Image.fromarray(resized)
#     img.show()

    dim = (64, 64)    
    resized = cv2.resize(img_dilation, dim, interpolation = cv2.INTER_AREA)
    
    return resized

In [29]:
#ta7a_17_7_100
t , h, e ,h_n , e_n = plot_hist("ta7a_14_7_100",0)
points_to_image("ta7a_14_7_100",0,t)

array([[0.        , 0.        , 0.        , ..., 0.        , 0.00124783,
        0.00341073],
       [0.        , 0.        , 0.        , ..., 0.0484316 , 0.04783335,
        0.13074448],
       [0.        , 0.        , 0.        , ..., 0.06476319, 0.03743479,
        0.10232177],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [30]:
################################# CNN #################################
# CNN Variables 
model_name = '/content/drive/MyDrive/models/waleed.h5'
ck_name = '/content/drive/MyDrive/models/cks/cp.ckpt'
batch_size = 64
epochs = 20
num_classes = 2
dim = 64
regularizer = 0.001
dropout = 0.25

In [ ]:
################################# CNN #################################
def create_2dimg_dataset():

    exp_p = [
        "waleed_60CM", 
        "nassar_50cm",
        "gedo_50cm",
        "gedo_14_7",
        "waleed_14_7",
        "ta7a_14_7",
        "nassar_14_7",
        "nassar_mask_glasses",
        "gedo_mask_glass",
        "gedo_bro2",
        "gedo_sis2",
        "gedo_sis1",
        "gedo_sis1_headphone",
        "gedo_mum",
        "gedo_bro1_mask_glass",
        "gedo_bro1",
        "waleed_noisy",
        "waleed_bad_ex",
        "ahmed_mum_40cm",
        "ahmed_mum2"
    ]

    exp_n = [
        "painting_50cm_rotation2",
        "painting_50cm_rotation",
        "painting_50cm",
        "painting_40cm_with_angle",
        "painting_40cm",
        "metal_dish",
        "painting", 
        "ahmed_image_40cm", 
        "ahmed_image_40cm_with_chair_behind", 
        "ahmed_image_50cm", 
        "ahmed_image_50cm_with_chair_behind", 
        "ahmed_image_60cm", 
        "ahmed_image_60cm_with_chair_behind", 
        "Ball_40cm", 
        "Ball_50cm", 
        "Ball_60cm", 
        "basket_40cm", 
        "basket_50cm", 
        "basket_60cm", 
        "vas",
        "air",
        "noisy_air",
        "wall_22_7"
            ]
    count = 0

    for i in range(len(exp_n)):
        feature_img = []
        target_list = []
        for j in range(1000):
            exp_name = exp_n[i] + "_" + str(j)
            t , h ,e , h_n, e_n= plot_hist(exp_name,1)
            img = points_to_image(exp_name,1,t)
            feature_img.append(img)
            target_list.append(0)
            count += 1
            print("count :: ",count)
        feature_img = np.array(feature_img)
        target_list = np.array(target_list)
        np.save("img_feature/64x64/"+exp_n[i]+"_features.npy",feature_img)
        np.save("img_feature/64x64/"+exp_n[i]+"_target_list.npy",target_list)

        
    for i in range(len(exp_p)):
        feature_img = []
        target_list = []
        for j in range(1000):
            exp_name = exp_p[i] + "_" + str(j)
            t , h ,e , h_n, e_n= plot_hist(exp_name,0)
            img = points_to_image(exp_name,0,t)
            feature_img.append(img)
            target_list.append(1)
            count += 1
            print("count :: ",count)
        feature_img = np.array(feature_img)
        target_list = np.array(target_list)
        np.save("img_feature/64x64/"+exp_p[i]+"_features.npy",feature_img)
        np.save("img_feature/64x64/"+exp_p[i]+"_target_list.npy",target_list)




In [ ]:
create_2dimg_dataset()

In [ ]:
################################# CNN #################################
# kernel_regularizer=regularizers.l1(regularizer)

# Initialising the CNN
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, (3, 3), input_shape = (dim,dim,1), activation = 'relu',kernel_regularizer=regularizers.l2(regularizer), bias_regularizer=regularizers.l2(regularizer)))
# model.add(BatchNormalization())
# Step 2 - Pooling
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(dropout))

# Adding a second convolutional layer
model.add(Conv2D(64, (3, 3), activation = 'relu' ,kernel_regularizer=regularizers.l2(regularizer), bias_regularizer=regularizers.l2(regularizer)))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(dropout))

# Adding a third convolutional layer
model.add(Conv2D(128, (3, 3), activation = 'relu',kernel_regularizer=regularizers.l2(regularizer), bias_regularizer=regularizers.l2(regularizer)))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(dropout))

# Adding a fourth convolutional layer
model.add(Conv2D(128, (3, 3), activation = 'relu',kernel_regularizer=regularizers.l2(regularizer), bias_regularizer=regularizers.l2(regularizer)))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(dropout))

# Step 3 - Flattening
model.add(Flatten())

# Step 4 - Full connection
model.add(Dense(units = 64, activation = 'relu' ,kernel_regularizer=regularizers.l2(regularizer), bias_regularizer=regularizers.l2(regularizer)))
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(Dense(units = 1, activation = 'sigmoid' ))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
################################# CNN #################################
model.summary()
# model.save(model_name)

In [ ]:

def shuffle_data(array1, array2):
    assert len(array1) == len(array2)
    p = np.random.permutation(len(array1))
    return array1[p], array2[p]
def generate_validation_set():
    exp_p = [
#             "waleed_60CM", 
#             "nassar_50cm",
#             "gedo_50cm",
#             "gedo_14_7",
#             "waleed_14_7",
#             "ta7a_14_7",
#             "nassar_14_7",
            "nassar_mask_glasses",
#             "gedo_mask_glass",
#             "gedo_bro2",
#             "gedo_sis2",
#             "gedo_sis1",
            "gedo_sis1_headphone"
            # "gedo_mum",
            # "gedo_bro1_mask_glass",
#             "gedo_bro1",
#             "waleed_noisy",
#             "waleed_bad_ex",
#             "ahmed_mum_40cm",
#             "ahmed_mum2"
    ]

    exp_n = [
#             "painting_50cm_rotation2",
#             "painting_50cm_rotation",
#             "painting_50cm",
#             "painting_40cm_with_angle",
#             "painting_40cm",
#             "metal_dish",
#             "painting", 
#             "ahmed_image_40cm", 
#             "ahmed_image_40cm_with_chair_behind", 
#             "ahmed_image_50cm", 
#             "ahmed_image_50cm_with_chair_behind", 
#             "ahmed_image_60cm", 
#             "ahmed_image_60cm_with_chair_behind", 
    #         "Ball_40cm", 
    #         "Ball_50cm", 
    #         "Ball_60cm", 
    #         "basket_40cm", 
    #         "basket_50cm", 
    #         "basket_60cm", 
    #         "vas",
            # "air",
            "noisy_air",
            "wall_22_7"
            ]


    count = 0
    size = max(len(exp_n),len(exp_p))
    total_data = []
    total_target_lists = []
    print ("Start Loading validation Data.")
    for i in range(size):

        if i < len(exp_n):
            data_list , target_list = np.load("img_feature/64x64/"+exp_n[i]+"_features.npy"),np.load("img_feature/64x64/"+exp_n[i]+"_target_list.npy")  
            # data_list = apply_transoformation(data_list)
            target_list = np.repeat(target_list, len(data_list)/ 1000)
            target_list = target_list 
            target_list = target_list.tolist()
            data_list = data_list.tolist()
            total_data.extend(data_list)
            total_target_lists.extend(target_list)

        if i < len(exp_p):
            data_list , target_list = np.load("img_feature/64x64/"+exp_p[i]+"_features.npy"),np.load("img_feature/64x64/"+exp_p[i]+"_target_list.npy")  
            # data_list = apply_transoformation(data_list)
            target_list = np.repeat(target_list, len(data_list)/ 1000)
            target_list = target_list.tolist()
            data_list = data_list.tolist()
            total_data.extend(data_list)
            total_target_lists.extend(target_list)
        count+= 1 
        print("Count :: ", count , ", Goal :: ",size)
    total_data = np.array(total_data)
    total_target_lists = np.array(total_target_lists)
    total_data = np.reshape(total_data,(-1, dim,dim, 1))
    total_data , total_target_lists = shuffle_data(total_data , total_target_lists)
    print("X validation data set :: ",total_data.shape)
    print("Y validation data set :: ",total_target_lists.shape)
    return total_data,total_target_lists


In [ ]:
x_valid , y_valid = generate_validation_set()

In [ ]:
################################# CNN #################################
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

def shuffle_data(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def apply_transoformation(imgs):
    num_generated = 4
    final_result =[]
    for img in imgs:
        img = img.reshape((1,1,) + img.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
        final_result.append(img)
        i = 0
        for batch in datagen.flow(img, batch_size=1):
            i += 1
            final_result.append(batch)
            
            if i == num_generated:
                break  # otherwise the ge
                
    final_result = np.array(final_result)
    final_result = final_result.reshape(-1,dim,dim)
    return final_result

def generate_test_set():
    exp_p = [
#             "waleed_60CM", 
#             "nassar_50cm",
#             "gedo_50cm",
#             "gedo_14_7",
#             "waleed_14_7",
#             "ta7a_14_7",
#             "nassar_14_7",
#             "nassar_mask_glasses",
#             "gedo_mask_glass",
#             "gedo_bro2",
#             "gedo_sis2",
#             "gedo_sis1",
#             "gedo_sis1_headphone",
            "gedo_mum",
            "gedo_bro1_mask_glass",
#             "gedo_bro1",
#             "waleed_noisy",
#             "waleed_bad_ex",
#             "ahmed_mum_40cm",
#             "ahmed_mum2"
    ]

    exp_n = [
#             "painting_50cm_rotation2",
#             "painting_50cm_rotation",
#             "painting_50cm",
#             "painting_40cm_with_angle",
#             "painting_40cm",
#             "metal_dish",
#             "painting", 
#             "ahmed_image_40cm", 
#             "ahmed_image_40cm_with_chair_behind", 
#             "ahmed_image_50cm", 
#             "ahmed_image_50cm_with_chair_behind", 
#             "ahmed_image_60cm", 
#             "ahmed_image_60cm_with_chair_behind", 
    #         "Ball_40cm", 
    #         "Ball_50cm", 
    #         "Ball_60cm", 
    #         "basket_40cm", 
    #         "basket_50cm", 
    #         "basket_60cm", 
    #         "vas",
            "air",
#             "noisy_air",
            "wall_22_7"
            ]


    count = 0
    size = max(len(exp_n),len(exp_p))
    total_data = []
    total_target_lists = []
    print ("Start Loading validation Data.")
    for i in range(size):

        if i < len(exp_n):
            data_list , target_list = np.load("img_feature/64x64/"+exp_n[i]+"_features.npy"),np.load("img_feature/64x64/"+exp_n[i]+"_target_list.npy")  
            data_list = apply_transoformation(data_list)
            target_list = np.repeat(target_list, len(data_list)/ 1000)
            target_list = target_list 
            target_list = target_list.tolist()
            data_list = data_list.tolist()
            total_data.extend(data_list)
            total_target_lists.extend(target_list)

        if i < len(exp_p):
            data_list , target_list = np.load("img_feature/64x64/"+exp_p[i]+"_features.npy"),np.load("img_feature/64x64/"+exp_p[i]+"_target_list.npy")  
            data_list = apply_transoformation(data_list)
            target_list = np.repeat(target_list, len(data_list)/ 1000)
            target_list = target_list.tolist()
            data_list = data_list.tolist()
            total_data.extend(data_list)
            total_target_lists.extend(target_list)
        count+= 1 
        print("Count :: ", count , ", Goal :: ",size)
    total_data = np.array(total_data)
    total_target_lists = np.array(total_target_lists)
    total_data = np.reshape(total_data,(-1, dim,dim, 1))
    total_data , total_target_lists = shuffle_data(total_data , total_target_lists)
    return total_data,total_target_lists

def train_CNN_model ():
    #waleed_mask_glass nassar_mask
    exp_p = [
            "waleed_60CM", 
            "nassar_50cm",
#             "gedo_50cm",
#             "gedo_14_7",
#             "waleed_14_7",
            "ta7a_14_7",
#             "nassar_14_7",
            "nassar_mask_glasses",
#             "gedo_mask_glass",
#             "gedo_bro2",
            "gedo_sis2",
#             "gedo_sis1",
#             "gedo_sis1_headphone",
#             "gedo_mum",
#             "gedo_bro1_mask_glass",
            "gedo_bro1",
            "waleed_noisy",
#             "waleed_bad_ex",
#             "ahmed_mum_40cm",
            "ahmed_mum2"
    ]

    exp_n = [
#             "painting_50cm_rotation2",
            "painting_50cm_rotation",
            "painting_50cm",
            "painting_40cm_with_angle",
#             "painting_40cm",
            "metal_dish",
#             "painting", 
            "ahmed_image_40cm", 
#             "ahmed_image_40cm_with_chair_behind", 
#             "ahmed_image_50cm", 
            "ahmed_image_50cm_with_chair_behind", 
#             "ahmed_image_60cm", 
            "ahmed_image_60cm_with_chair_behind", 
    #         "Ball_40cm", 
    #         "Ball_50cm", 
    #         "Ball_60cm", 
    #         "basket_40cm", 
    #         "basket_50cm", 
    #         "basket_60cm", 
    #         "vas",
#             "air",
            "noisy_air",
#             "wall_22_7"
            ]


    count = 0
    size = max(len(exp_n),len(exp_p))
    total_data = []
    total_target_lists = []
    print ("Start Loading Training Data.")
    for i in range(size):

        if i < len(exp_n):
            data_list , target_list = np.load("img_feature/64x64/"+exp_n[i]+"_features.npy"),np.load("img_feature/64x64/"+exp_n[i]+"_target_list.npy")  
            data_list = apply_transoformation(data_list)
            target_list = np.repeat(target_list, len(data_list)/ 1000)
#             print(target_list)
            target_list = target_list 
            target_list = target_list.tolist()
            data_list = data_list.tolist()
            total_data.extend(data_list)
            total_target_lists.extend(target_list)

        if i < len(exp_p):
            data_list , target_list = np.load("img_feature/64x64/"+exp_p[i]+"_features.npy"),np.load("img_feature/64x64/"+exp_p[i]+"_target_list.npy")  
            data_list = apply_transoformation(data_list)
            target_list = np.repeat(target_list, len(data_list)/ 1000)
#             print(target_list)
            target_list = target_list.tolist()
            data_list = data_list.tolist()
            total_data.extend(data_list)
            total_target_lists.extend(target_list)
        count+= 1 
        print("Count :: ", count , ", Goal :: ",size)
    total_data = np.array(total_data)
    total_target_lists = np.array(total_target_lists)
    total_data = np.reshape(total_data,(-1, dim,dim, 1))
    #     total_target_lists = np.reshape(total_target_lists,(-1, 300,300, 1))
    #     total_target_lists = np.reshape(total_target_lists , (-1,1))
    # print(total_data.shape)
    total_data , total_target_lists = shuffle_data(total_data , total_target_lists)
#     print(total_data.shape)


#     X_train, X_test, y_train, y_test = train_test_split(total_data, total_target_lists, train_size=0.8,test_size=0.2, random_state=101) # 70% training and 30% test
    X_train , y_train = total_data,total_target_lists
    X_test , y_test = generate_test_set()
    model = load_model(model_name)
    print ("Start Training.")
    fashion_train = model.fit(X_train, y_train, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test, y_test))
    model.save(model_name)


    print("Training is Done.")
    

In [ ]:
train_CNN_model ()

In [26]:
################################# CNN #################################
##fully trained model#################################### 
model =tf.keras.models.load_model("cnn/final_model_all_data.h5")


In [36]:

exp_name = "test_waleed2_17_7"
folder_indx = 0
def nn_pred(img):
    img = np.reshape(img,(1,64,64, 1))
    result = model.predict(img)
    print("result :: ",result[0][0])
    return result[0][0] > 0.5 

def pred_one(exp_name,folder_indx):
    print("exp_name :: ",exp_name)
    t , h ,e , h_n, e_n= plot_hist(exp_name,folder_indx)
    img = points_to_image(exp_name,folder_indx,t)
    print("is this 3D object :: " ,nn_pred(img))
    
def pred_many(exp_name,folder_indx):
    print("exp_name :: ",exp_name)
    result_p = []
    result_n = []
    count = 0
    for i in range(1000):
        t , h ,e , h_n, e_n= plot_hist(exp_name+"_"+str(i),folder_indx)
        img = points_to_image(exp_name+"_"+str(i),folder_indx,t)
        count +=1
        print("count :: ",count)
        y_pred = nn_pred(img)
        if y_pred == True or y_pred == 1:
            result_p.append(y_pred)
        else:
            result_n.append(y_pred)

            
    print("number of positive results :: ", len(result_p))
    print("number of negative results :: ", len(result_n))
    print("positive accuracy :: ", (100 *len(result_p)/(len(result_p)+len(result_n)) ))
    print("negative accuracy :: ", (100 *len(result_n)/(len(result_p)+len(result_n)) ))
    
pred_one(exp_name,folder_indx)
# pred_many(exp_name,folder_indx)

exp_name ::  test_waleed2_17_7
result ::  0.95388615
is this 3D object ::  True


In [ ]:
################################# CNN #################################


# NN Method

In [ ]:
################################################## NN ################################################################
from tensorflow import keras
from tensorflow.keras import layers,Sequential,models

X_train, X_test, y_train, y_test = train_test_split(total_data[:,0:199], total_data[:,199], train_size=0.7,test_size=0.3, random_state=101) # 70% training and 30% test
model = keras.Sequential()
model.add(layers.Dense(32,input_dim=199,activation='relu'))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(8,activation='relu'))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test))

In [ ]:
################################################## NN ################################################################
def nn_pred(hist):
    result = model.predict(np.reshape(hist,(-1,199)))
    return result[0][0] > .5

In [ ]:
################################################## NN ################################################################
t , h, e , h_n , e_n = plot_hist("test_waleed2_17_7",0)
f = (h_n - np.min(h_n)) /(np.max(h_n) - np.min(h_n))
data_list = np.array([f])
res = nn_pred(data_list)
print(res)

# Generate 1000 Sample from 1 Sample By Adding Randome Noise

In [ ]:
import random


exp_name = "test_waleed2_17_7"

dist = np.loadtxt("3D_Experements"+"/"+exp_name+"_x.txt")
upper_angle = np.loadtxt("3D_Experements"+"/"+exp_name+"_y.txt")
lower_angle = np.loadtxt("3D_Experements"+"/"+exp_name+"_z.txt")




#x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)
for i in range(1000):
#     n = random.randint(5,20)
    new_dist = dist
#     for j in range(n):
#         k = random.randint(0,5)
#         l = random.randint(0,len(dist) - 1)
#         new_dist[l] = new_dist[l] + k
        
    file_data = open("3D_Experements/waleed_bad_ex_"+str(i)+"_x.txt", "a")
    np.savetxt(file_data, new_dist)
    file_data.close()
    
    file_data = open("3D_Experements/waleed_bad_ex_"+str(i)+"_y.txt", "a")
    np.savetxt(file_data, upper_angle)
    file_data.close()
    
    file_data = open("3D_Experements/waleed_bad_ex_"+str(i)+"_z.txt", "a")
    np.savetxt(file_data, lower_angle)
    file_data.close()
    
    print(i)
        

    

# Unwanted Cells

In [ ]:
b = np.load("img_feature/negative/wall_22_7_features.npy")
print(b.shape)
f = b[0]

print(f.shape)
# img = Image.fromarray(f)
# img.show()

b = np.reshape(b,(-1, 300,300, 1))
print(b.shape)
f2 =b[0,:,:,:]
print(f2.shape)
cv2.imshow("Resized image", f2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def plot_3D (exp_name , depth_axis):
    dist = np.loadtxt("3D_Experements/"+exp_name+"_x.txt")
    upper_angle = np.loadtxt("3D_Experements/"+exp_name+"_y.txt")
    lower_angle = np.loadtxt("3D_Experements/"+exp_name+"_z.txt")

    print(np.amax(upper_angle*180/np.pi))
    print(np.amin(upper_angle*180/np.pi))
    print(np.amax(lower_angle*180/np.pi))
    print(np.amin(lower_angle*180/np.pi))
    

    #x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)

    my_sample_x = np.array(dist)*np.cos(upper_angle)*np.sin(lower_angle)
    my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
    my_sample_z =  np.array(dist)*np.sin(upper_angle)
    
    my_sample_x = my_sample_x[~np.isnan(my_sample_x)]
    my_sample_y = my_sample_y[~np.isnan(my_sample_y)]
    my_sample_z = my_sample_z[~np.isnan(my_sample_z)]

    df = pd.DataFrame(my_sample_x,columns=['X (mm)'])
    df['Y (mm)'] = my_sample_y
    df['Z (mm)'] = my_sample_z
    df.head()
    min_depth = 0
    max_depth = 0
    if depth_axis == 'x':
        df['Depth'] = my_sample_x
        max_depth = np.amax(my_sample_x)
        min_depth = np.amin(my_sample_x)

    elif depth_axis == 'y':
        df['Depth'] = my_sample_y
        max_depth = np.amax(my_sample_y)
        min_depth = np.amin(my_sample_y)

    elif depth_axis == 'z':
        df['Depth'] = my_sample_z
        max_depth = np.amax(my_sample_z)
        min_depth = np.amin(my_sample_z)
        

    else:
        print("invalid depth axis")
        return
   

    fig = px.scatter_3d(df, x='X (mm)', y='Y (mm)', z='Z (mm)', color='Depth', title="ٌRadar Point Cloud" , range_color=[min_depth-200,max_depth+200],color_continuous_scale=color , opacity=1)
    fig.update_traces(marker=dict(size=marker_size, line=dict(width=0))) 
    fig.show()


def plot_2D (exp_name):
    dist = np.loadtxt("3D_Experements/"+exp_name+"_x.txt")
    upper_angle = np.loadtxt("3D_Experements/"+exp_name+"_y.txt")
    lower_angle = np.loadtxt("3D_Experements/"+exp_name+"_z.txt")

    print(np.amax(upper_angle*180/np.pi))
    print(np.amin(upper_angle*180/np.pi))
    print(np.amax(lower_angle*180/np.pi))
    print(np.amin(lower_angle*180/np.pi))
    

    #x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)

    my_sample_x = np.array(dist)*np.cos(upper_angle)*np.sin(lower_angle)
    my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
    my_sample_z =  np.array(dist)*np.sin(upper_angle)

    minx = np.min(my_sample_x)
    miny = np.min(my_sample_y)
    minz = np.min(my_sample_z)

    maxx = np.max(my_sample_x)
    maxy = np.max(my_sample_y)
    maxz = np.max(my_sample_z)

    # img =np.zeros(int(maxx-minx),int(maxz-minz))
    # img =np.zeros((int(maxx-minx),int(maxz-minz)), dtype=np.float64)
    # img =np.zeros(len(my_sample_x),len(my_sample_z)), dtype=np.float64)

    # for i in range(len(my_sample_x) - 1):
    #     img[int(my_sample_x[i] - minx -1)][int(my_sample_z[i] - minz -1)] = (my_sample_y[i] / maxy)*255
    
    trace = go.Surface(x = my_sample_x, y = my_sample_y, z =my_sample_z )
    data = [trace]
    layout = go.Layout(title = '3D Surface plot')
    fig = go.Figure(data = data)
    plot(fig)
    offline.init_notebook_mode(connected=False)

    df3 = {'x':[1, 2, 3, 4, 5],'y':[10, 20, 30, 40, 50],'z': [[5, 4, 3, 2, 1]]*5}
    offline.iplot(dict(data=[go.Surface(x=df3['x'], y=df3['y'], z=df3['z'])]))

def plot_var(threshold):
    exp_p = ["gedo_14_7",
             "waleed_14_7",
             "ta7a_14_7",
             "nassar_14_7"]

    exp_n = ["metal_dish",
            "painting"]

    var_all = []
    count = 0
    color = []

    for i in range(len(exp_p)):
        for j in range(1000):
            exp_name = exp_p[i] + "_" + str(j)
            dist = np.loadtxt("3D_Experements/"+exp_name+"_x.txt")
            upper_angle = np.loadtxt("3D_Experements/"+exp_name+"_y.txt")
            lower_angle = np.loadtxt("3D_Experements/"+exp_name+"_z.txt")
            my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)

            min_depth = np.amin(my_sample_y)
            max_depth = (min_depth + threshold)
            indx = my_sample_y <= (min_depth + threshold)
            my_sample_y = my_sample_y[indx]
            var_all.append(np.var(my_sample_y))
            count+=1
            color.append(0)
            # print(count)

    
    for i in range(len(exp_n)):
        for j in range(1000):
            exp_name = exp_n[i] + "_" + str(j)
            dist = np.loadtxt("flat_Experements/"+exp_name+"_x.txt")
            upper_angle = np.loadtxt("flat_Experements/"+exp_name+"_y.txt")
            lower_angle = np.loadtxt("flat_Experements/"+exp_name+"_z.txt")
            my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
            # var_all.append(np.var(my_sample_y))

            min_depth = np.amin(my_sample_y)
            max_depth = (min_depth + threshold)
            indx = my_sample_y <= (min_depth + threshold)
            my_sample_y = my_sample_y[indx]
            var_all.append(np.var(my_sample_y))
            color.append(1)
            count+=1
            # print(count)

    x = [num for num in range(0, len(var_all), 1)]
    # a = np.array([x,var_all])

    categories = np.array(color)

    colormap = np.array(['g', 'r'])

    plt.scatter(np.array(x), np.array(var_all), s=100, c=colormap[categories])

    # plt.savefig('ScatterClassPlot.png')
    # plt.show()
     
    
    # plt.plot(x, var_all,'o',color = color)
    plt.xlabel('X')
    plt.ylabel('var')
    plt.grid(True)

    plt.plot()